In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q scipy
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This b

In [ ]:
model_id = "bigscience/bloom-560m"

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto', trust_remote_code=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
text = "Loài hoa nào phổ biến được dùng để trang trí trong dịp Tết ở miền Bắc Việt Nam? "
input_ids = tokenizer.encode(text, return_tensors='pt').to("cuda:0")
sample_outputs = model.generate(input_ids, pad_token_id=50256,
                                   do_sample=True,
                                   max_length=100,
                                   top_p=90,
                                   top_k=50,
                                   temperature=0.8,
                                   early_stopping=False,
                                   no_repeat_ngram_size=2,
                                   num_return_sequences=3)

for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))
    print('\n---')

>> Generated text 1

Loài hoa nào phổ biến được dùng để trang trí trong dịp Tết ở miền Bắc Việt Nam?  Đất nước Việt được thiên nhiên ưu ái nên người ta bắt đầu trồng nhiều loại cây như đào, cúc, thanh long, xoan tây, vv. Ngoài ra, người dân ở Bắc bộ thường trồng những loài cây cảnh khác như cây dừa, cây xoài, những cây gỗ, cành cây, chồi cây và cành lá của những loại thảo dược như thuốc bắc, đỗ xuyên, g

---
>> Generated text 2

Loài hoa nào phổ biến được dùng để trang trí trong dịp Tết ở miền Bắc Việt Nam?  Tết vừa rồi đi đâu mà không ghé thăm thăm ở đây một lần?
Bạn đang tìm kiếm một địa điểm lưu trú để tận hưởng những khoảnh khắc đẹp trong ngày Tết của mình ở Miền Bắc? Hãy ghé ngay khu biệt thự nghỉ dưỡng tại Phú Quốc để nghỉ ngơi những ngày cuối tuần. Với hơn 100 căn biệt lập độc đáo trên đảo Phú quốc, bạn sẽ được hòa mình vào không

---
>> Generated text 3

Loài hoa nào phổ biến được dùng để trang trí trong dịp Tết ở miền Bắc Việt Nam?  Hình ảnh này được đăng tải trên mạng xã hội 

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
!pip install datasets
from datasets import load_dataset

data = load_dataset ("thanhnew2001/newyear", split='train')


data = data.map(
    lambda samples: {
        "input_text": '###Human: ' + samples["question"] + '###Assisstant: ' + samples["answer"] + ' ### '
    }
, remove_columns= ['question', 'answer'])

data = data.map(lambda examples: tokenizer(examples["input_text"]), batched=True)


#data = data.map(lambda x: tokenizer(f"{x['question']}"), batched=True)

Generating train split:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
import transformers
import torch.cuda
import torch

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=10,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=500,
        learning_rate=2e-4,
        fp16=False,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
1,1.307800
2,1.307800
3,1.244800
4,1.182100
5,1.127000
6,1.072700
7,1.015600
8,0.955000
9,0.897800
10,0.837400


TrainOutput(global_step=500, training_loss=0.05404249326884746, metrics={'train_runtime': 303.3347, 'train_samples_per_second': 65.934, 'train_steps_per_second': 1.648, 'total_flos': 336440893440000.0, 'train_loss': 0.05404249326884746, 'epoch': 500.0})

In [ ]:
trainer.save_model('adapter-model')


In [ ]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
        model_id,
        return_dict=True,
        torch_dtype=torch.float16,
        trust_remote_code=True
)

model = PeftModel.from_pretrained(base_model, 'adapter-model').to('cuda')
print(f"Running merge_and_unload")
model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.save_pretrained("merged_model")
tokenizer.save_pretrained("merged_model")


Running merge_and_unload


('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/tokenizer.json')

In [ ]:
double_quant_nf4_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant=True
)
model_id ='merged_model'
model_double = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=double_quant_nf4_config, device_map="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
text = "###Human: Loài hoa nào phổ biến được dùng để trang trí trong dịp Tết ở miền Bắc Việt Nam? ###Assistant: "
input_ids = tokenizer.encode(text, return_tensors='pt').to("cuda:0")
sample_outputs = model.generate(input_ids, pad_token_id=50256,
                                   do_sample=True,
                                   max_length=100,
                                   top_p=90,
                                   top_k=50,
                                   temperature=0.8,
                                   early_stopping=False,
                                   no_repeat_ngram_size=2,
                                   num_return_sequences=1)

for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))
    print('\n---')

>> Generated text 1

###Human: Loài hoa nào phổ biến được dùng để trang trí trong dịp Tết ở miền Bắc Việt Nam? ###Assistant: 兰花 ### 川山 ###

Human 答 卜: 金牛 ###   ###### 川县 ### ### 人 目: 牛 ###
###### ### 牛###
川山县 ### 金 牛 百度名片 ### 注 义： 牛</s>

---
